# Загрузка и обучение модели общей болталки

## Загрузим необходимые библиотеки

In [1]:
import os
import numpy as np
import pandas as pd

import mmap
import re
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

from sklearn.model_selection import train_test_split

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

from tqdm.notebook import tqdm

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
tqdm.pandas()
device = torch.device("cuda")

## Определим необходимые функции и классы

In [3]:
# Функция возвращает количество строк в текстовом файле
def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [4]:
# Функция предобработки текста
def preprocess_txt(line):
    # Удаляем HTML теги
    spls = re.sub('<[^<]+?>', ' ', line)
    # Удаляем URLs
    spls = re.sub(r'http\S+', ' ', spls)
    # Удаляем \t табуляцию
    spls = re.sub('\t', ' ', spls)
    # Убираем специальные символы: избавляемся от всего, что не является "словами"
    spls = re.sub('[^a-zA-Zа-яА-ЯёЁ0-9\s!?-]', ' ', spls)
    # Удаляем многократное повторение букв
    spls = re.sub(r"\а+", 'а', spls)
    spls = re.sub(r"\у+", 'у', spls)
    spls = re.sub(r"\к+", 'к', spls)
    spls = re.sub(r"\ы+", 'ы', spls)
    # Убираем лишние пробелы
    spls = re.sub(r"\s\s+", ' ', spls)
    spls = spls.lower()
    if not spls:
        spls = np.nan
    return spls

In [5]:
def build_train_val_data(q_data, a_data):
    output_data = []
    for idx in tqdm(range(q_data.shape[0])):
        question = q_data.iloc[idx]
        answer = a_data.iloc[idx]
        output_data.append("\nq:" + question + "\na:" + answer)
    return output_data

In [6]:
class CommonChatterDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_text):
        self.tokenized_text = tokenized_text
        
    def __getitem__(self, idx):
        item = {q: a[idx].clone().detach() for q, a in self.tokenized_text.items()}
        return item
    
    def __len__(self):
        return len(self.tokenized_text["input_ids"])

In [7]:
# Функция генерирующая ответ на вопрос пользователя
def respond_to_dialog(texts, model, tokenizer):
    prefix = '\nq:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nq:' if i % 2 == 1 else '\na:'
    #print(prefix)
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [8]:
# Функция запускающая и поддерживающая диалог с пользователем
def start_dialog(tokenizer, model):
    seed = input('Начните диалог с ботом любой фразой\n Для завершения диалога введите: Stop\n')
    history = [seed]
    is_continue_dilog = True
    
    while is_continue_dilog:
        if seed == 'Stop':
            break
            
        output = respond_to_dialog([seed], model, tokenizer)
        seed = input(output + '\n')
        
        history.append(output)
        history.append(seed)
    return history

## Загрузка и предобработка текста вопросов-ответов для болталки

In [9]:
# Преобразуем файл вопросов ответов в два списка вопросов и ответов
# при этом будем создавать пару вопрос + и первый ответ
data_q = []
data_a = []

is_question = False
is_answer = False

file_path_from = "Otvety.txt"

n_line = get_num_lines(file_path_from)
with open("Otvety.txt", "r", encoding="UTF-8") as fin:
    for i in tqdm(range(n_line)):
        line = fin.readline()
        if line.startswith("---"):
            is_question = True
            is_answer = False
            continue
        elif is_question and (not is_answer):
            question = line.strip()
            is_answer = True
            continue
        elif is_question and is_answer:
            answer = line.strip()
            
            data_q.append(question)
            data_a.append(answer)
            
            is_question = False
            is_answer = False

  0%|          | 0/7550926 [00:00<?, ?it/s]

Сохраним вопросы и ответы в датафрейм

In [11]:
data_qa = pd.DataFrame(data_q[:10000], columns=['question'])
data_qa['answer'] = data_a[:10000]
data_qa.shape

(10000, 2)

In [12]:
data_qa.head(3)

,question,answer
0,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,хомячка....
1,Как парни относятся к цветным линзам? Если у д...,меня вобще прикалывает эта тема :).
2,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свали..."


Предобработаем текст

In [13]:
data_qa['question'] = data_qa['question'].progress_apply(preprocess_txt)
data_qa['answer'] = data_qa['answer'].progress_apply(preprocess_txt)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

Удаляем строки с пустым ответом или вопросом

In [14]:
data_qa.dropna(inplace=True)
data_qa.reset_index(inplace=True)
data_qa.drop(labels=['index'], axis=1, inplace=True)

In [15]:
data_qa.shape

(10000, 2)

In [16]:
print("Максимальное количество слов в вопросах: {} и ответах: {}"
      .format(np.max([len(text) for text in data_qa['question']]), np.max([len(text) for text in data_qa['answer']])))
print("Среднее количество слов в вопросах: {} и ответах: {}"
      .format(np.mean([len(text) for text in data_qa['question']]), np.mean([len(text) for text in data_qa['answer']])))
print("Минимальное количество слов в вопросах: {} и ответах: {}"
      .format(np.min([len(text) for text in data_qa['question']]), np.min([len(text) for text in data_qa['answer']])))

Максимальное количество слов в вопросах: 3947 и ответах: 45756
Среднее количество слов в вопросах: 125.5285 и ответах: 308.3286
Минимальное количество слов в вопросах: 1 и ответах: 1


Добавляем метку класса

In [17]:
data_qa["class"] = 1

In [18]:
data_qa.head()

,question,answer,class
0,вопрос о тдв давно и хорошо отдыхаем лично вам...,хомячка,1
1,как парни относятся к цветным линзам? если у д...,меня вобще прикалывает эта тема,1
2,что делать сегодня нашёл 2 миллиона рублей?,если это счастье действительно на вас свалилос...,1
3,эбу в двенашке называется итэлма что за эбу?,эбу электронный блок управления двигателем авт...,1
4,академия вампиров сколько на даный момент част...,4 охотники и жертвы ледяной укус поцелуй тьмы ...,1


Сохраним датафрейм в csv

In [19]:
data_qa.to_csv('common_talker_data_clean.csv')

#### Разобьем ответы на обучающую и тестовую выборки

In [20]:
%%time
train_q, test_q, train_a, test_a = train_test_split(data_qa.question, data_qa.answer, test_size=0.2, random_state=21)

CPU times: total: 0 ns
Wall time: 2.29 ms


In [21]:
print("Размер обучающей выборки: "+ str(len(train_q)))
print("Размер тестовой выборки: "+ str(len(test_q)))

Размер обучающей выборки: 8000
Размер тестовой выборки: 2000


#### Преобразуем обучающие и тестовые данные для токенизации

In [22]:
train_dataset = build_train_val_data(train_q, train_a)
test_dataset = build_train_val_data(test_q, test_a)

  0%|          | 0/8000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

#### Выполним токенизацию обучающей и тестовой выборок

In [23]:
sber_model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer_sd = AutoTokenizer.from_pretrained(sber_model_name)

In [24]:
tokenizer_sd.pad_token = tokenizer_sd.eos_token

In [25]:
%%time
tokenized_train_dataset = tokenizer_sd(train_dataset, padding="max_length", truncation=True, max_length=512, return_tensors='pt')
tokenized_test_dataset = tokenizer_sd(test_dataset, padding="max_length", truncation=True, max_length=512, return_tensors='pt')

CPU times: total: 14.9 s
Wall time: 4.24 s


#### Преобразуем в стандартный датасет torch

In [26]:
train_dataset_pt = CommonChatterDataset(tokenized_train_dataset)
test_dataset_pt = CommonChatterDataset(tokenized_test_dataset)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer_sd, mlm=False)

## Загрузка и дообучение модели болталки

### Загрузка предобученой модели GPT3 от SberDevices

In [27]:
model_sd = AutoModelForCausalLM.from_pretrained(sber_model_name).to(device)

### Дообучение модели на наших данных

#### Определяем аргументы для Trainer

In [28]:
training_args = TrainingArguments(
    output_dir="gpt2-sd-up", # Выходной каталог
    overwrite_output_dir=True, # устанавливаем перезапись конткнта в выходном каталоге
    num_train_epochs=4, # nколичество эпох обучения
    per_device_train_batch_size=3, # размер батча для обучения
    per_device_eval_batch_size=4,  # размер батча для проверки
    eval_steps = 400, #  Количество шагов между оценками
    save_steps=800, # полсе # шага модель сохраняется
    warmup_steps=500,# количество шагов для планировщика шага обучения
    )

#### Создаем Trainer

In [29]:
trainer = Trainer(
    model=model_sd,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset_pt,
    eval_dataset=test_dataset_pt
)

#### Производим дообучение и сохраняем полученную модель и токенайзер

In [30]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,4.056400
1000,4.042000
1500,3.998700
2000,4.007700
2500,3.990300
3000,3.628900
3500,3.418600
4000,3.518300
4500,3.479700
5000,3.479600


TrainOutput(global_step=10668, training_loss=3.3130955848078805, metrics={'train_runtime': 1700.8922, 'train_samples_per_second': 18.814, 'train_steps_per_second': 6.272, 'total_flos': 8361345024000000.0, 'train_loss': 3.3130955848078805, 'epoch': 4.0})

In [36]:
trainer.save_model('trainer_ct_model')

In [33]:
tokenizer_sd.save_pretrained('final_ct_gpt_tk')
model_sd.save_pretrained('final_ct_gpt_model')

### Загрузка модели и ткенайзера и проверка работы

In [34]:
tokenizer_fsd = AutoTokenizer.from_pretrained("final_ct_gpt_tk")
model_fsd = AutoModelForCausalLM.from_pretrained("final_ct_gpt_model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [35]:
start_dialog(tokenizer_fsd, model_fsd)

Начните диалог с ботом любой фразой
 Для завершения диалога введите: Stop
 Привет!
ты кто?
 Гость
уважаемый гость! мы не виделись с вами это может быть связано с какими-то делами?
 Да был в командировке
был!!! а что там нового???
 Что делать если я нашел миллион рублей?
выиграй в лотерею и купи себе новый айфон!
 Тебе наравятся цветные линзы?
вроде нет!!! но думаю что не исключено!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 Чем ты сейчас занимаешся?
спать ложусь!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!!
 Спокойной ночи!
доброго!
 Stop


['Привет!',
 'ты кто?',
 'Гость',
 'уважаемый гость! мы не виделись с вами это может быть связано с какими-то делами?',
 'Да был в командировке',
 'был!!! а что там нового???',
 'Что делать если я нашел миллион рублей?',
 'выиграй в лотерею и купи себе новый айфон!',
 'Тебе наравятся цветные линзы?',
 'вроде нет!!! но думаю что не исключено!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 'Чем ты сейчас занимаешся?',
 'спать ложусь!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!! ы!!!',
 'Спокойной ночи!',
 'доброго!',
 'Stop']